**Full Name: shayan aryania**

**St ID: 402211767**

#Expectation Maximization Algorithm

This EM algorithm implementation is designed to uncover hidden patterns in multiple DNA sequences, with a focus on identifying optimal motif locations and alignment positions. As an unsupervised learning technique, the EM algorithm is well-suited for discovering latent variables in complex data sets. The program's primary objectives are to provide a clear and efficient implementation of the EM algorithm and to facilitate the discovery of meaningful motifs and alignment positions in DNA sequences.

The motif locations and alignment positions are latent variables that require inference, as they are known to exist but cannot be directly observed. To address this challenge, the program employs a iterative approach, refining its estimates of these variables until convergence is achieved.

A key feature of this implementation is that the initial motif positions are randomly initialized, which means that each run of the program will produce unique results. However, the program's ability to consistently produce similar results when provided with the same input demonstrates its robustness.

The Expectation step is the algorithm's workhorse, responsible for calculating positional frequencies, odds ratios, and log-odds scores. These scores are then used to derive the log-likelihood function, which is optimized in the Maximization step. By iteratively refining its estimates of the motif positions and alignment scores, the program is able to identify the most likely motifs and alignment positions in the input DNA sequences.

### Install and import required packages

In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter

### Load dataset file
Loads all user data including the number of random alignments, the number of iterations, and the FASTA file.

In [21]:
motif_len = 6
random_alignment_total = 200
iteration = 200

In [20]:
with open("example.fasta") as fasta:
    sequences = [x.strip("\n") for i, x in enumerate(fasta.readlines()) if i % 2 == 1]

### Prepare data
Centralizes all relevant and reusable data, encompassing a comprehensive range of information. This includes the concatenation of all bases into a single string, a tally of individual base counts, an inventory of contiguous motifs, and the calculation of cumulative sums.

In [ ]:
all_base_chars = "".join(sequences)

total_base_frequency = Counter(all_base_chars)

seqs_len = [len(x)- motif_len for x in sequences]
seqs_len.insert(0, 0)

initial_cumulative_sum = np.array(seqs_len)
cumulative_sum_sequence = np.cumsum(initial_cumulative_sum)

motif_contigs = np.array([list(seq[start:start + motif_len])
            for seq in sequences
            for start in range(len(seq) - motif_len)])

## EM Steps:

In [14]:
def initialize_random_positions(seqs_len):
    motif_start_pos = [random.randint(0, length) for length in seqs_len]
    return motif_start_pos

In [15]:
def motif_count_EM(motif_start_pos, sequences, total_base_frequency, motif_len):
    em_motif = [x[pos:pos + motif_len] for x, pos in zip(sequences, motif_start_pos)]
    motif_base_chars = "".join(em_motif)

    total_bkg_counts = total_base_frequency - Counter(motif_base_chars)

    motif_posits_freqs = {base: np.ones(motif_len) for base in 'ACGT'}
    for i in range(motif_len):
        for base in 'ACGT':
            motif_posits_freqs[base][i] += motif_base_chars[i::motif_len].count(base)

    return total_bkg_counts, motif_posits_freqs

In [16]:
def generate_EM_matrix(total_counts, motif_pos_freq, motif_len):
    em_log_odds_ratio = np.empty((4, motif_len))

    total_bases = sum(total_counts.values())
    background_freq = {base: total_counts[base] / total_bases for base in 'ACGT'}

    for i in range(motif_len):
        for j, base in enumerate('ACGT'):
            motif_freq = motif_pos_freq[base][i]
            bkg_freq = background_freq[base]
            odds_ratio = motif_freq / bkg_freq
            em_log_odds_ratio[j, i] = np.log(odds_ratio)

    return em_log_odds_ratio

In [17]:
def base_index(base):
    return {'A': 0, 'C': 1, 'G': 2, 'T': 3}[base]

In [18]:
def calculate_EM_score(em_log_odds_ratio, motif_len, sequences, iteration, total_base_frequency):
    for _ in range(iteration):
        motif_start_posits = []
        for sequence in sequences:
            max_score = -np.inf
            max_posit = 0
            for i in range(len(sequence) - motif_len + 1):
                score = sum(em_log_odds_ratio[base_index(sequence[i + j])][j] for j in range(motif_len))
                if score > max_score:
                    max_score = score
                    max_posit = i
            motif_start_posits.append(max_posit)

        total_counts, motif_pos_freq = motif_count_EM(motif_start_posits, sequences, total_base_frequency, motif_len)
        em_log_odds_ratio = generate_EM_matrix(total_counts, motif_pos_freq, motif_len)

    max_likely_results = {"Final Positions": motif_start_posits, "Final Scores": [], "Final Motifs": []}
    for i, sequence in enumerate(sequences):
        final_pos = motif_start_posits[i]
        final_motif = sequence[final_pos:final_pos + motif_len]
        final_score = sum(em_log_odds_ratio[base_index(base)][j] for j, base in enumerate(final_motif))
        max_likely_results["Final Scores"].append(final_score)
        max_likely_results["Final Motifs"].append(final_motif)

    return max_likely_results


The entire process iterates until convergence is achieved, The optimal results from each iteration are then documented for future reference.

In [19]:
def run_em_algorithm(sequences, motif_len, iteration, random_alignment_total):
    total_base_chars = "".join(sequences)
    total_base_frequency = Counter(total_base_chars)
    seqs_len = [len(x) - motif_len for x in sequences]

    total_records = []
    for _ in range(random_alignment_total):
        motif_start_pos = initialize_random_positions(seqs_len)
        total_counts, motif_pos_freq = motif_count_EM(motif_start_pos, sequences, total_base_frequency, motif_len)
        em_log_odds_ratio = generate_EM_matrix(total_counts, motif_pos_freq, motif_len)
        maximum_likelihood = calculate_EM_score(em_log_odds_ratio, motif_len, sequences, iteration, total_base_frequency)
        
        maximum_likelihood["Max Final Score"] = max(maximum_likelihood["Final Scores"])
        maximum_likelihood["Max Sum Scores"] = sum(maximum_likelihood["Final Scores"])
        max_final_sequence_idx = maximum_likelihood["Final Scores"].index(maximum_likelihood["Max Final Score"])
        maximum_likelihood["Max Final Sequence"] = max_final_sequence_idx
        maximum_likelihood["Max Final Position"] = maximum_likelihood["Final Positions"][max_final_sequence_idx]
        maximum_likelihood["Max Final Motif"] = maximum_likelihood["Final Motifs"][max_final_sequence_idx]
        
        total_records.append(maximum_likelihood)

    final_results = max(total_records, key=lambda x: x["Max Sum Scores"])
    return final_results


In [22]:
final_results = run_em_algorithm(sequences, motif_len, iteration, random_alignment_total)

The scoring results for the aligned sequences, and the maximum statistics are displayed.

In [23]:
final_dataframe = pd.DataFrame({key:final_results[key] for key in ("Final Scores", "Final Positions", "Final Motifs")})

del final_results["Final Positions"]
del final_results["Final Scores"]
del final_results["Final Motifs"]

print(final_dataframe)
print(*final_results.items(), sep='\n')

    Final Scores  Final Positions Final Motifs
0      26.728441               18       TTATCA
1      24.377937               22       CGGTCA
2      26.202749               15       CTATCA
3      23.632581               17       AGATAA
4      23.705029               16       TGATTA
5      25.992735               18       CTATCT
6      26.728441               20       TTATCA
7      26.728441                2       TTATCA
8      25.698765               17       CTATAA
9      25.992735               14       CTATCT
10     24.903629               21       TGGTCA
11     25.409777               33       TTGTAA
12     26.518427               20       TTATCT
13     26.518427                2       TTATCT
14     26.728441               10       TTATCA
15     25.698765                3       CTATAA
16     24.399645               21       TGGTAA
17     23.422566               20       AGATAT
18     25.214326               16       TGATAA
19     23.632581               24       AGATAA
20     24.674